In [87]:
import pickle
import pandas as pd
import sklearn.preprocessing as skp
from sklearn.decomposition import SparsePCA
from sklearn.decomposition import MiniBatchSparsePCA
from sklearn.decomposition import TruncatedSVD

In [3]:
reducedWordFeatures = pickle.load(open("../WikiTextScraper/FinalWordFeatures.p","rb"))

In [114]:
reducedWordFeatures.shape

(83297, 1391)

In [129]:
import pymysql

# your username and password go here!
creds = {'user': 'brian', 
         'pswd': 'farley'} 


connection = pymysql.connect(host="atriskwiki.cbprs6vpqcbz.us-west-1.rds.amazonaws.com",
                            user=creds['user'],
                            password=creds['pswd'],
                            port=3306,
                            db="atriskwiki",
                            charset="utf8")

cur = connection.cursor()

In [9]:
basicInfo = "SELECT page_id,page_is_ep FROM core WHERE page_in_train = 1 AND page_batch < 99"
cur.execute(basicInfo)

trainPages = pd.DataFrame(list(cur.fetchall()),columns=['page_id','page_is_ep'])

In [10]:
trainPages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62491 entries, 0 to 62490
Data columns (total 2 columns):
page_id       62491 non-null int64
page_is_ep    62491 non-null int64
dtypes: int64(2)
memory usage: 976.5 KB


In [58]:
trainPageIDs = list(trainPages['page_id'])
epTrainPageIDs = list(trainPages[trainPages['page_is_ep'] == 1]['page_id'])
len(epTrainPageIDs)

5682

In [59]:
trainWordFeatures = reducedWordFeatures[reducedWordFeatures.index.isin(trainPageIDs)]
epTrainWordFeatures = reducedWordFeatures[reducedWordFeatures.index.isin(epTrainPageIDs)]
colmax = trainWordFeatures.max(axis=0)
rowmax = trainWordFeatures.max(axis=1)
ep_rowmax = epTrainWordFeatures.max(axis=1)
trainWordFeatures = trainWordFeatures[rowmax > 0]
epTrainWordFeatures = epTrainWordFeatures[ep_rowmax > 0]

In [60]:
trainWordFeatures.shape
epTrainWordFeatures.shape

(5561, 1391)

In [61]:
features = list(trainWordFeatures.columns)
epfeatures = list(epTrainWordFeatures.columns)

In [62]:
import scipy.sparse
twf_array = scipy.sparse.csr_matrix(trainWordFeatures.values)
etwf_array = scipy.sparse.csr_matrix(epTrainWordFeatures.values)

In [63]:
maxScaler = skp.MaxAbsScaler(copy=False)
maxScaler.fit_transform(twf_array)
maxScaler.fit_transform(etwf_array)

<5561x1391 sparse matrix of type '<class 'numpy.float64'>'
	with 165498 stored elements in Compressed Sparse Row format>

In [28]:
makeFeatures = TruncatedSVD(n_components=10)
makeFeatures.fit(twf_array)

TruncatedSVD(algorithm='randomized', n_components=10, n_iter=5,
       random_state=None, tol=0.0)

In [29]:
makeFeatures.explained_variance_ratio_

array([ 0.05732398,  0.02794786,  0.01860229,  0.01260526,  0.00959564,
        0.00845628,  0.0078254 ,  0.00785554,  0.0075793 ,  0.0069186 ])

In [38]:
SVD_components = makeFeatures.components_
type(SVD_components)
type(trainWordFeatures.columns.values)

numpy.ndarray

In [45]:
SVD_comps = pd.DataFrame(SVD_components.T)

In [47]:
SVD_comps['fnames'] = features

In [49]:
SVD_comps.to_csv(path_or_buf="SVD_components.csv")

In [50]:
ep_features = TruncatedSVD(n_components=10)

In [64]:
ep_features.fit(etwf_array)

TruncatedSVD(algorithm='randomized', n_components=10, n_iter=5,
       random_state=None, tol=0.0)

In [65]:
ep_features.explained_variance_ratio_

array([ 0.0260936 ,  0.00565139,  0.00761925,  0.0067588 ,  0.00610145,
        0.005827  ,  0.00524086,  0.00464094,  0.00430182,  0.00424544])

In [67]:
epSVD_comps = pd.DataFrame(ep_features.components_.T)
epSVD_comps['fnames'] = epfeatures

In [68]:
epSVD_comps.to_csv(path_or_buf="epSVD_components.csv")

In [81]:
epSparse = SparsePCA(n_components=50,n_jobs=6,alpha=0.3)
epSparse.fit(etwf_array.toarray())

/Users/brian/.pyenv/versions/3.5.1/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/Users/brian/.pyenv/versions/3.5.1/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/Users/brian/.pyenv/versions/3.5.1/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is depre

SparsePCA(U_init=None, V_init=None, alpha=0.3, max_iter=1000, method='lars',
     n_components=50, n_jobs=6, random_state=None, ridge_alpha=0.01,
     tol=1e-08, verbose=False)

In [96]:
epSparsePCA = pd.DataFrame(epSparse.components_.T)
epSparsePCA['fnames'] = epfeatures

KeyError: '0'

In [83]:
epSparsePCA.to_csv(path_or_buf="epSparsePCA_50_comps.csv")

In [100]:
with open("PCA_components_ep.txt","w") as f:
    for i in range(50):
        pca_comp = epSparsePCA[['fnames',i]]
        pca_comp.columns=['fword','component']
        pca_comp = pca_comp[pca_comp['component'] != 0].sort_values('component',axis=0)
        f.write("component {}\n".format(i))
        f.write("============\n")
        for x in zip(list(pca_comp['fword']),list(pca_comp['component'])):
            f.write("{}: {:.3f}\n".format(x[0],x[1]))
        f.write("\n")

In [101]:
epSparsePCA.shape

(1391, 51)

In [110]:
rRowMax = reducedWordFeatures.max(axis=1)
rwfWithWords = reducedWordFeatures[rRowMax > 0]
page_ids = rwfWithWords.index
PCAwords = epSparse.transform(rwfWithWords.as_matrix())

In [126]:
colnames = ['lgbt','playoffs','places','israel','rich_people','animals','soccer','criticism','motorsports',
           'pop_music','egypt','dogs','diseases','acting','islam','scientology','crimes','sports_championships',
           'highways','bad','college_sports','apologies','pornography','south_hemisphere','reality_tv','philanthropy',
           'melissa_armstrong','towns','austria','suspicious_museum','seeds','non_us_sports','sad_olympians',
           'west_bank','disney_movies','tv_for_teens','boats','basketball','sunk_warships','gymnastics','barack_obama',
           'behavior','graphing','verdicts','interest','internet_communication','eastern_europe','white_people',
           'sections','islam_in_pakistan']
frmtCols = ["pca_{}".format(x) for x in colnames]
PCA_features = pd.DataFrame(PCAwords)
PCA_features.columns = frmtCols
PCA_features['page_id'] = list(page_ids)

In [128]:
records_for_db = PCA_features.to_dict(orient='records')

In [138]:
connection.commit()

In [139]:
createPCA = """CREATE TABLE PCA_word_features (
             page_id INT UNSIGNED NOT NULL,
             {},
             PRIMARY KEY (page_id)
             ) DEFAULT CHARACTER SET utf8"""
colSpec = "{} FLOAT"
cols = [colSpec.format(x) for x in frmtCols]
cur.execute(createPCA.format(",".join(cols)))

0

In [131]:
def chunk(l,n):
    for i in range(0,len(l),n):
        yield(l[i:i+n])

In [140]:
PCA_insert = "INSERT INTO PCA_word_features({}) VALUES {}"
dbChunks = chunk(records_for_db,500)
for rChunk in dbChunks:
    cols = list(rChunk[0].keys())
    insertData = []
    for record in rChunk:
        insertData.append("({})".format(",".join([str(x) for x in list(record.values())] ) ) )
    insertme = PCA_insert.format(",".join(cols),",".join(insertData))
    cur.execute(insertme)
    connection.commit()

In [142]:
zero_insert = "INSERT IGNORE INTO PCA_word_features({}) VALUES {}"
cur.execute("SELECT page_id FROM core WHERE page_batch < 99")
pids = list(cur.fetchall())
cols = "page_id," + ",".join(frmtCols)
zeros = ['0'] * 50
insertvals = "({},{})"
for idchunk in chunk(pids,2000):
    inserts = []
    for pid in idchunk:
        inserts.append(insertvals.format(pid,",".join(zeros)))
    #cur.execute(zero_insert.format(cols,",".join(inserts)))
    connection.commit()

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,' at line 1")